In [1]:
import pathlib
import subprocess
import importlib
import sys

if not importlib.util.find_spec("mypy"):
    subprocess.run([sys.executable,"-m","pip","install","mypy"])

In [2]:
a = 10
match a:
    case int():
        print("a is of integer type")
    case float():
        print("a is of float type")


a is of integer type


In [3]:
path = "/root"
match path:
    case str():
        path = pathlib.Path(path)
    case pathlib.Path():
        path = path.name

type(path)

pathlib.PosixPath

In [4]:
match path:
    case str():
        path = pathlib.Path(path)
    case pathlib.Path():
        path = path.name

type(path)

str

In [5]:
paths = ["/root","/usr"]
match paths:
    case str():
        paths = [pathlib.Path(paths)]
    case list(str()):
        paths = list(map(pathlib.Path,paths))
    case pathlib.Path():
        paths = [paths]
paths

['/root', '/usr']

In [6]:
def test_match(some):
    match some:
        case str():
            print("it is string")
        case list() as list_some if all(isinstance(a,str) for a in list_some):
            print("list of string")
        case list() as list_some if all(isinstance(a,pathlib.Path) for a in list_some):
            print("list of Path")
        case _:
            print("dont know")

In [7]:
test_match("hi")

it is string


In [8]:
test_match(["g","g"])

list of string


In [9]:
test_match(list(map(pathlib.Path,list("hello"))))

list of Path


In [10]:
%%writefile test_convert_to_list_paths.py
import pathlib

def convert_to_list_paths(
    paths: pathlib.Path | str | list[str],
) -> list[pathlib.Path] | None:
    match paths:
        case str():
            return [pathlib.Path(paths)]
        case pathlib.Path():
            return [paths]
        case list() as list_str if all(isinstance(a, str) for a in list_str):
            return list(map(pathlib.Path, list_str))
        case _:
            return None

convert_to_list_paths(pathlib.Path("a"))
convert_to_list_paths("a")
convert_to_list_paths(list("hello"))
list_path:list[pathlib.Path] = list(map(lambda x :pathlib.Path(x),list("hello")))
# convert_to_list_paths(list_path)

Overwriting test_convert_to_list_paths.py


In [11]:
mypy_out = subprocess.run(["mypy","test_convert_to_list_paths.py"],capture_output=True,text=True)
print(mypy_out.returncode)
print(mypy_out.stdout)

0
Success: no issues found in 1 source file



In [12]:
from test_convert_to_list_paths import convert_to_list_paths

path_dict = {"source_path": ["/root","/etc"], "out_path": "/usr"}
match path_dict:
    case {
        "source_path": list(src_paths) | str(src_paths) | pathlib.Path(src_paths),
        "out_path": str(out_paths) | list(out_paths) | pathlib.Path(out_paths)
        }:
        path_dict = {"source_path": convert_to_list_paths(src_paths), 
                     "out_path": convert_to_list_paths(out_paths)}
    case _:
        raise TypeError("str|list(str)|Path")

In [13]:
path_dict

{'source_path': [PosixPath('/root'), PosixPath('/etc')],
 'out_path': [PosixPath('/usr')]}